# Data Clean and Extract Matrices
In this notebook, we load in the CSV file of the raw_data and clean up the data and extract three different matrices from the cleaned data.

Change csv_filename and which_dataset.
Then run all cells.

You will get three npz files:
1. rating_matrix_fantasy.npz: a sparse matrix of the rating matrix
2. isRead_matrix_fantasy.npz: a sparse matrix of the isRead matrix
3. shelved_matrix_fantasy.npz: a sparse matrix of the shelved matrix
*you can use to_dense() function on these matrices to make them into a numpy format

In additional you'll also get two other files:
1. BOOK_ID_TO_INT_poetry.json: a dictionary mapping book_id in
   five_review_three_column_rating_poetry to ints starting from 0
2. USER_ID_TO_INT_poetry.json: a dictionary mapping user_id in
   five_review_three_column_rating_poetry to ints starting from 0

## Load the Raw Data

In [3]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import sys
import json
import gc

In terminal, issue python2.7 json_to_csv_converter.py goodreads_interactions_fantasy_paranormal.json
substitute goodreads_interactions_fantasy_paranormal.json with any json dataset
you will get goodreads_interactions_fantasy.csv

In [1]:
csv_filename = "goodreads_interactions_fantasy_paranormal.csv"
which_dataset = "fantasy"

In [4]:
raw_df = pd.read_csv(csv_filename)

In [5]:
raw_df.head(3)

,user_id,started_at,date_added,read_at,book_id,date_updated,rating,isRead,review_id
0,8842281e1d1347389f2ab93d60773d4d,NaN,Fri Sep 08 10:44:24 -0700 2017,NaN,19161852,Fri Sep 08 10:44:24 -0700 2017,0,False,b109f004e43b53456d363f93abe1c5c1
1,8842281e1d1347389f2ab93d60773d4d,Tue Aug 15 13:23:18 -0700 2017,Sun Jul 30 07:44:10 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,18245960,Wed Aug 30 00:00:26 -0700 2017,5,True,b35cd7abfa362ed5a0c246ccac450967
2,8842281e1d1347389f2ab93d60773d4d,NaN,Wed May 31 06:41:50 -0700 2017,NaN,32075825,Wed May 31 06:41:51 -0700 2017,0,False,6b45b65a7ecbba0114bf50855ce4f46b


In [6]:
raw_df.columns

Index(['user_id', 'started_at', 'date_added', 'read_at', 'book_id',
       'date_updated', 'rating', 'isRead', 'review_id'],
      dtype='object')

In [7]:
# keep only three columns: rating, user_id, book_id

In [8]:
raw_df = raw_df[['user_id','book_id','rating','isRead' ]]

In [9]:
#convert isRead to 0 and 1 for our isRead matrix later
raw_df.isRead = raw_df.isRead.astype("int8")

In [10]:
raw_df.head(5)

,user_id,book_id,rating,isRead
0,8842281e1d1347389f2ab93d60773d4d,19161852,0,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,5,1
2,8842281e1d1347389f2ab93d60773d4d,32075825,0,0
3,8842281e1d1347389f2ab93d60773d4d,43615,0,0
4,8842281e1d1347389f2ab93d60773d4d,26721984,0,0


In [13]:
raw_df.shape

(55506178, 4)

In [14]:
# number of unique users and number of unique books

In [15]:
raw_df.user_id.unique().shape

(726932,)

In [16]:
raw_df.book_id.unique().shape

(258585,)

In [17]:
raw_df.rating.unique()

array([0, 5, 3, 4, 2, 1])

In [18]:
# use a new reference
rating_df = raw_df
raw_df = None
gc.collect()

0

## Query Only Books with Over 500 Ratings

To help our model, we only query books with over 500 numbers of ratings so that there are enough interactions for us to predict the actual ratings.

In [19]:
# add a new column rating_counts of each book to rating_df
rating_counts = rating_df.book_id.value_counts(sort=False)
rating_df["rating_counts"] = rating_df.book_id.apply(lambda id: rating_counts[id])

In [20]:
# only keep books that have larger or equal to 500 ratings
rating_df = rating_df.query("rating_counts >= 500")

In [21]:
# rating_counts colomn is not useful anymore, drop it
rating_df.drop("rating_counts", axis=1, inplace=True)

In [22]:
# how many books left
len(rating_df["book_id"].unique())

14473

In [23]:
# how many users left
len(rating_df["user_id"].unique())

706391

In [24]:
rating_df["rating"].unique()

array([0, 5, 3, 4, 2, 1])

## Delete Duplicate Users-Book Pairs from Raw Data

Although the UCSD data source claims that the data is already cleaned and ready to use, we found out that there are actually repeated user-book pairs in the dataset. This has caused our initial models to have very high MSE's. To deal with this problem, we decided to delete the duplicate user-book pairs from our data as part of our data cleaning process.

In [25]:
# Problem: there are repeated user-book pairs in the dataset!
# Solution: just delete all rows of those users

users = rating_df["user_id"].tolist()
books = rating_df["book_id"].tolist()

print (str((len(users), len(books))))

(42073486, 42073486)


In [26]:
pairs = []

for index in range(len(users)):
    pairs.append((users[index], books[index]))
    
users = None
books = None
gc.collect()

0

In [27]:
dupicate_pairs_set = set()
seen = set()

for pair in pairs:
    if pair not in seen:
        seen.add(pair)
    else:
        dupicate_pairs_set.add(pair)
    
pairs = None
seen = None
gc.collect()

0

In [28]:
# number of duplicated pairs
len(dupicate_pairs_set)

74518

In [29]:
duplicate_users = set([user_book_pair[0] for user_book_pair in dupicate_pairs_set])
len(duplicate_users)

27278

In [30]:
rating_df.shape

(42073486, 4)

In [31]:
# remove all rows of duplicated users
rating_df = rating_df[~rating_df["user_id"].isin(duplicate_users)]
duplicate_users = None
gc.collect()

0

In [32]:
rating_df.shape

(35284388, 4)

In [33]:
len(rating_df["user_id"].unique())

679113

In [34]:
len(rating_df["book_id"].unique())

14473

## Map User-id from String to Int

The original user_id is a string type. We want this to be an int as we can use it as an index in our sparse matrices. Below is our way of mapping from user_id string to user_id int. We save a file called USER_ID_TO_INT.json to help us retrieve back these mappings.

In [35]:
# user_id is string, but we need it to be int
rating_df.dtypes

user_id    object
book_id     int64
rating      int64
isRead       int8
dtype: object

In [36]:
# solution: map each user_id to an integer starting from 0

USER_ID_TO_INT = {}
COUNTER = 0

def map_user_to_int(user_id):
    global USER_ID_TO_INT
    global COUNTER
    
    if user_id not in USER_ID_TO_INT:
        USER_ID_TO_INT[user_id] = COUNTER
        COUNTER += 1
        
rating_df["user_id"].apply(map_user_to_int)
USER_ID_TO_INT

{'8842281e1d1347389f2ab93d60773d4d': 0,
 '72fb0d0087d28c832f15776b0d936598': 1,
 'ab2923b738ea3082f5f3efcbbfacb218': 2,
 'd986f354a045ffb91234e4af4d1b12fd': 3,
 '7504b2aee1ecb5b2872d3da381c6c91e': 4,
 'f8a89075dc6de14857561522e729f82c': 5,
 '47d4a49ae9ee68ae78d891b3a259b695': 6,
 '704eb93a316aff687a93d5215882eb21': 7,
 '06316bec7a49286f1f98d5acce24f923': 8,
 '012515e5802b2e0f42915118c90fa04b': 9,
 'fe0df2eff573e75c036eb8287c6b012a': 10,
 '1711b2a40d154603f157536619f71967': 11,
 'f4d16ea4ac59af59d257631398af39f4': 12,
 '220ef9c058a2132e6a9827f93a821d87': 13,
 '01ec1a320ffded6b2dd47833f2c8e4fb': 14,
 '4b3636a043e5c99fa27ac897ccfa1151': 15,
 '4035e5f05352217609c1a294410f2d50': 16,
 '903d4b859e86a1dd6d7640849cc7067c': 17,
 'afc070543f19028dc7e7f084a0079f72': 18,
 'd92c94bda1ca3ec254f3aa95757c7831': 19,
 'dc3763cdb9b2cae805882878eebb6a32': 20,
 '4980305f36ab8c2ab831e401a185f28a': 21,
 '96a4eae3201cf9eb0fd36a900611c925': 22,
 '7b2e5fe9fd353fecf3eeebb4850b88d3': 23,
 'bafc2d50014200cda7cb2b6a

In [37]:
# now we can convert each user_id from a string to an int

rating_df["user_id"] = rating_df["user_id"].apply(lambda user_id: USER_ID_TO_INT[user_id])
rating_df.head(3)

,user_id,book_id,rating,isRead
0,0,19161852,0,0
1,0,18245960,5,1
2,0,32075825,0,0


In [38]:
rating_df.dtypes

user_id    int64
book_id    int64
rating     int64
isRead      int8
dtype: object

In [39]:
# number of unique users is same as before
len(rating_df.user_id.unique())

679113

In [40]:
rating_df["rating"].unique()

array([0, 5, 3, 4, 2, 1])

## Map Book-id from 0 to Total Number of Books

The original book_id is an int type. However, it is not a continuous count of int. This becomes messy and difficult for us to use as index in our sparse matrix. We then do the same as we did for the user_id. We mapped these book_id's to a range of int and saved a file BOOK_ID_TO_INT.json to help us retrieve back this information.

In [41]:
# now lets convert book_id to consecutive ints beginning from 0

COUNTER = 0
BOOK_ID_TO_INT = {}

def map_book_to_int(id):
    global BOOK_ID_TO_INT
    global COUNTER
    
    if id not in BOOK_ID_TO_INT:
        BOOK_ID_TO_INT[id] = COUNTER
        COUNTER += 1
    
rating_df["book_id"].apply(map_book_to_int)
BOOK_ID_TO_INT

{19161852: 0,
 18245960: 1,
 32075825: 2,
 43615: 3,
 13543159: 4,
 8138071: 5,
 428263: 6,
 29570143: 7,
 10893214: 8,
 16174968: 9,
 17554595: 10,
 18423: 11,
 29058155: 12,
 186074: 13,
 64222: 14,
 27003: 15,
 15839976: 16,
 76852: 17,
 7562764: 18,
 908296: 19,
 12111823: 20,
 5577844: 21,
 17315048: 22,
 13642: 23,
 8908: 24,
 61886: 25,
 13531802: 26,
 13453029: 27,
 13239822: 28,
 3973: 29,
 10572: 30,
 62291: 31,
 862041: 32,
 33: 33,
 40866: 34,
 17238: 35,
 6984678: 36,
 6065215: 37,
 28876: 38,
 47212: 39,
 41804: 40,
 16690: 41,
 2213661: 42,
 136251: 43,
 104342: 44,
 142296: 45,
 76620: 46,
 34497: 47,
 18512: 48,
 15241: 49,
 34: 50,
 11: 51,
 14497: 52,
 77566: 53,
 18122: 54,
 119322: 55,
 33507: 56,
 7082: 57,
 52357: 58,
 1622: 59,
 5907: 60,
 12067: 61,
 37684: 62,
 18131: 63,
 11129: 64,
 1226: 65,
 106: 66,
 103: 67,
 6: 68,
 2: 69,
 4: 70,
 3: 71,
 1: 72,
 30: 73,
 12: 74,
 15881: 75,
 7840190: 76,
 518848: 77,
 37442: 78,
 16793: 79,
 9460487: 80,
 13496: 81,
 

In [42]:
# convert book_id to consecutive int

rating_df["book_id"] = rating_df["book_id"].apply(lambda book_id: BOOK_ID_TO_INT[book_id])
rating_df.head(3)

,user_id,book_id,rating,isRead
0,0,0,0,0
1,0,1,5,1
2,0,2,0,0


In [43]:
# save the two mappings to disk as json because we may need them later
# you will get USER_ID_TO_INT_poetry.json
# and BOOK_ID_TO_INT_poetry.json

with open("USER_ID_TO_INT_" + which_dataset + ".json", "w+") as f:
    json.dump(USER_ID_TO_INT, f)
    
with open("BOOK_ID_TO_INT_" + which_dataset + ".json", "w+") as f:
    json.dump(BOOK_ID_TO_INT, f)


In [44]:
USER_ID_TO_INT = None
BOOK_ID_TO_INT = None
gc.collect()

64

In [45]:
# number of unique books is same as before
len(rating_df.book_id.unique())

14473

In [46]:
rating_df["rating"].unique()

array([0, 5, 3, 4, 2, 1])

In [47]:
# finally we can convert rating_df to a sparse rating matrix
sparse_rating_matrix = sp.coo_matrix((rating_df["rating"], (rating_df["user_id"], rating_df["book_id"])))

In [48]:
# number of rows is correctly number of unique users
# number of columns is correctly number of unique books
sparse_rating_matrix.shape

(679113, 14473)

In [49]:
set(sp.find(sparse_rating_matrix.data)[2])

{1, 2, 3, 4, 5}

## Save the Three Matrices as npz files

Here we save the matrices as a sparse npz file (as opposed to numpy files). This helps us save some storage.

In [50]:
# save sparse matrix to disk
# you will get clean_sparse_rating_matrix_poetry.npz
sp.save_npz("rating_matrix_" + which_dataset, sparse_rating_matrix)

In [51]:
sparse_rating_matrix = sp.coo_matrix((rating_df["isRead"], (rating_df["user_id"], rating_df["book_id"])))

In [52]:
sp.save_npz("isRead_matrix_" + which_dataset, sparse_rating_matrix)

In [53]:
rating_df['shelved'] = 1

In [54]:
sparse_rating_matrix = sp.coo_matrix((rating_df["shelved"], (rating_df["user_id"], rating_df["book_id"])))

In [ ]:
sp.save_npz("shelved_matrix_" + which_dataset, sparse_rating_matrix)